In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [2]:
# Load the data
data = pd.read_csv('Churn_Modelling.csv')


In [3]:
print(data.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

***Preprocessing***

In [4]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

**Encoding categorical**

In [5]:
label_encoder_geography = LabelEncoder()
label_encoder_gender = LabelEncoder()

data['Geography'] = label_encoder_geography.fit_transform(data['Geography'])
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [6]:
geography_mapping = dict(zip(label_encoder_geography.classes_, label_encoder_geography.transform(label_encoder_geography.classes_)))
gender_mapping = dict(zip(label_encoder_gender.classes_, label_encoder_gender.transform(label_encoder_gender.classes_)))

print("Geography Mapping:", geography_mapping)
print("Gender Mapping:", gender_mapping)

Geography Mapping: {'France': 0, 'Germany': 1, 'Spain': 2}
Gender Mapping: {'Female': 0, 'Male': 1}


**Scaling**

In [7]:
# Select features and target
X = data.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = data['Exited']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


**Train_Test_Split**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


**Build and Train the Neural Network**

In [9]:
# Build the neural network
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
250/250 [==============================] - 4s 7ms/step - loss: 0.5938 - accuracy: 0.7189 - val_loss: 0.4850 - val_accuracy: 0.8055
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5015 - accuracy: 0.7941 - val_loss: 0.4469 - val_accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4769 - accuracy: 0.7960 - val_loss: 0.4340 - val_accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4676 - accuracy: 0.7981 - val_loss: 0.4247 - val_accuracy: 0.8040
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4528 - accuracy: 0.7999 - val_loss: 0.4166 - val_accuracy: 0.8070
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4508 - accuracy: 0.8043 - val_loss: 0.4109 - val_accuracy: 0.8120
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4455 - accuracy: 0.8071 - val_loss: 0.4040 - val_accuracy: 0.8205

**Test Metrics**

In [10]:
# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


63/63 [==============================] - 0s 1ms/step
Accuracy: 0.8595
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.97      0.92      1607
           1       0.77      0.40      0.53       393

    accuracy                           0.86      2000
   macro avg       0.82      0.69      0.72      2000
weighted avg       0.85      0.86      0.84      2000

Confusion Matrix:
 [[1560   47]
 [ 234  159]]


**Make prediction**

In [11]:
import numpy as np

# Function to take user input, preprocess, and predict churn
def predict_churn():
    # Collect user input
    print("Enter the details of the customer:")
    credit_score = float(input("Credit Score: "))
    geography = input("Geography (France/Spain/Germany): ")
    gender = input("Gender (Male/Female): ")
    age = int(input("Age: "))
    tenure = int(input("Tenure: "))
    balance = float(input("Balance: "))
    num_of_products = int(input("Number of Products: "))
    has_cr_card = int(input("Has Credit Card (0/1): "))
    is_active_member = int(input("Is Active Member (0/1): "))
    estimated_salary = float(input("Estimated Salary: "))

    # Preprocess the input
    geography_encoded = label_encoder_geography.transform([geography])[0]
    gender_encoded = label_encoder_gender.transform([gender])[0]

    # Create a DataFrame for the input
    input_data = pd.DataFrame({
        'CreditScore': [credit_score],
        'Geography': [geography_encoded],
        'Gender': [gender_encoded],
        'Age': [age],
        'Tenure': [tenure],
        'Balance': [balance],
        'NumOfProducts': [num_of_products],
        'HasCrCard': [has_cr_card],
        'IsActiveMember': [is_active_member],
        'EstimatedSalary': [estimated_salary]
    })

    # Scale the input data
    input_scaled = scaler.transform(input_data)

    # Predict using the trained model
    prediction = model.predict(input_scaled)

    # Output the prediction
    churn_prob = prediction[0][0]
    print(f"Churn Probability: {churn_prob:.2f}")
    if churn_prob > 0.5:
        print("Prediction: The customer is likely to churn.")
    else:
        print("Prediction: The customer is not likely to churn.")

# Call the function to predict churn
predict_churn()


Enter the details of the customer:
Credit Score: 616
Geography (France/Spain/Germany): France
Gender (Male/Female): Male
Age: 69
Tenure: 10
Balance: 619987
Number of Products: 4
Has Credit Card (0/1): 1
Is Active Member (0/1): 0
Estimated Salary: 10198
1/1 [==============================] - 0s 24ms/step
Churn Probability: 1.00
Prediction: The customer is likely to churn.
